In [1]:
import pandas as pd
import numpy as np


In [2]:
places_df = pd.read_csv('f2.csv')

In [3]:
places_df.head()

,id,main_places,sub_places,sub2_places,rating,time
0,Sportsathlete,"Munnar, India","Chithirapuram, India",Chandy's Windy Woods Spa,5,January 2020
1,Travellinguli,"Munnar, India","Chithirapuram, India",Chandy's Windy Woods Spa,5,December 2019
2,123Margaret4711,"Munnar, India","Chithirapuram, India",Chandy's Windy Woods Spa,5,November 2019
3,SKG109,"Munnar, India","Pothamedu, India",Photo Point,3,February 2020
4,eric,"Munnar, India","Pothamedu, India",Photo Point,3,January 2020


In [4]:
combine_rating=places_df.dropna(axis=0,subset=['sub2_places'])
places_ratingcount =(combine_rating.
                    groupby(by=['sub2_places'])['rating'].
                    count().
                    reset_index().
                    rename(columns={'rating':'totalratingcount'})
                    [['sub2_places','totalratingcount']]
                    )

In [5]:
places_ratingcount.head()

,sub2_places,totalratingcount
0,Abad Nucleus Mall,5
1,Adi Shancharachaya Temple,4
2,Alova Hospitality,4
3,Aluva Shiva Temple,8
4,Alwaye Palace,8


In [6]:
df=combine_rating.merge(places_ratingcount,left_on='sub2_places',right_on='sub2_places',how='left')
df.head()

,id,main_places,sub_places,sub2_places,rating,time,totalratingcount
0,Sportsathlete,"Munnar, India","Chithirapuram, India",Chandy's Windy Woods Spa,5,January 2020,3
1,Travellinguli,"Munnar, India","Chithirapuram, India",Chandy's Windy Woods Spa,5,December 2019,3
2,123Margaret4711,"Munnar, India","Chithirapuram, India",Chandy's Windy Woods Spa,5,November 2019,3
3,SKG109,"Munnar, India","Pothamedu, India",Photo Point,3,February 2020,3
4,eric,"Munnar, India","Pothamedu, India",Photo Point,3,January 2020,3


In [7]:
"""
popularity_threshold = 5
df=df.query('totalratingcount >= @popularity_threshold')
df.head()
"""

"\npopularity_threshold = 5\ndf=df.query('totalratingcount >= @popularity_threshold')\ndf.head()\n"

In [8]:
df.shape

(480, 7)

In [9]:
#Creationg of Pivot table
features_df=df.pivot_table(index='sub2_places',columns='id',values='rating').fillna(0)
features_df.head()

id,0703Vas63,123Margaret4711,19PankajS94,23MZ68,34Nina,36 Palms,AJoooooii,Abdur Rahman,Adhik,Aflalrahman,...,tomas s,travel961,travelbugsy7,venguru,vinita s,vinoopm,vmsatheesh22,webbin,yesarar,അനൂപ് V
sub2_places,,,,,,,,,,,,,,,,,,,,,
Abad Nucleus Mall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adi Shancharachaya Temple,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alova Hospitality,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aluva Shiva Temple,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alwaye Palace,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#converting the table to array matrix
from scipy.sparse import csr_matrix
features_df_matrix=csr_matrix(features_df.values)



In [11]:
from sklearn.neighbors import NearestNeighbors

model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [12]:
query_index=np.random.choice(features_df.shape[0])
print(query_index)

15


In [13]:
distances,indices=model_knn.kneighbors(features_df.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)

In [17]:
print(indices,' : ',distances)

[[15 64 65 62 63 60]]  :  [[0. 1. 1. 1. 1. 1.]]


In [18]:
for i in range(0,len(distances.flatten())):
    
    if i==0:
        print('Places which are similar to {}:\n'.format(features_df.index[query_index]))    
    else:
        print('{0}:{1}, with distance of {2}.'.format(i,features_df.index[indices.flatten()[i]],distances.flatten()[i]))

Places which are similar to Blossom Hydel Park:

1:Periyakanal Waterfalls, with distance of 1.0.
2:Photo Point, with distance of 1.0.
3:Paavakkulam Mahadeva Temple, with distance of 1.0.
4:Pack Your Passport, with distance of 1.0.
5:Neptune Water Sports, with distance of 1.0.
